# Testing models

In [49]:
import sys
sys.path.append('src/')
import pandas as pd
from Models import deberta_base_nli, bart_nli, deberta_v3_nli
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from Helpers import *

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

## Basic tests of models

In [2]:
deberta_base_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

({'contradiction': 96.9, 'neutral': 3.0, 'entailment': 0.0}, ['contradiction'])

In [3]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

({'contradiction': 93.7, 'neutral': 5.8, 'entailment': 0.5}, ['contradiction'])

In [4]:
deberta_v3_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

/usr/local/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


({'contradiction': 97.7, 'neutral': 2.2, 'entailment': 0.1}, ['contradiction'])

# Testing models with random samples from toxic dataset

In [53]:
df = get_random_samples("data/toxicbias_train.csv", 50)

## Calculate labels

In [47]:
import pandas as pd

def add_nli_predictions(df, model):
    df = df.copy()  # Create a copy of the DataFrame

    df = df.apply(lambda row: predict_labels(row, model), axis=1)

    return df

def predict_labels(row, model):
    comment_text = row['comment_text']
    bias = row['bias']
    rationale = row['rationale']

    if bias == 'neutral' or pd.isna(rationale):
        rationale = "Bias or prejudice in the text."

    scores, labels = model(comment_text, rationale)
    row['predicted_label_rationale'] = labels
    row['predicted_scores_rationale'] = scores

    scores, labels = model(comment_text, "Bias or prejudice in the text.")
    row['predicted_label_base'] = labels
    row['predicted_scores_base'] = scores

    return row

In [104]:
dfs = {
    'deberta-base': add_nli_predictions(df, deberta_base_nli),
    'bart-large': add_nli_predictions(df, bart_nli),
    'deberta-v3': add_nli_predictions(df, deberta_v3_nli)
}

/usr/local/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Breakdown of labels 

In [129]:
def label_breakdown(df, label_column_name = 'predicted_label_rationale'):
    label_breakdown = {
        'bias': {'contradiction': 0, 'neutral': 0, 'entailment': 0},
        'neutral': {'contradiction': 0, 'neutral': 0, 'entailment': 0}
    }

    for _, row in df.iterrows():
        bias = row['bias']
        predicted_label = row[label_column_name][0]
        
        label_breakdown[bias][predicted_label] += 1

    return label_breakdown


In [171]:
## compare results of rationale hypothesis and base one
results = {}

for model, data in dfs.items():
  results[model] = {
      'rationale hypothesis':label_breakdown(data),
      'base hypothesis':label_breakdown(data, 'predicted_label_base'),
    }
  
for model, data in results.items():
  print(model)
  print(data['rationale hypothesis'])
  print(data['base hypothesis'])
  print()


deberta-base
{'bias': {'contradiction': 3, 'neutral': 34, 'entailment': 3}, 'neutral': {'contradiction': 0, 'neutral': 1, 'entailment': 9}}
{'bias': {'contradiction': 3, 'neutral': 4, 'entailment': 33}, 'neutral': {'contradiction': 0, 'neutral': 1, 'entailment': 9}}

bart-large
{'bias': {'contradiction': 6, 'neutral': 29, 'entailment': 5}, 'neutral': {'contradiction': 0, 'neutral': 1, 'entailment': 9}}
{'bias': {'contradiction': 1, 'neutral': 4, 'entailment': 35}, 'neutral': {'contradiction': 0, 'neutral': 1, 'entailment': 9}}

deberta-v3
{'bias': {'contradiction': 1, 'neutral': 34, 'entailment': 5}, 'neutral': {'contradiction': 1, 'neutral': 7, 'entailment': 2}}
{'bias': {'contradiction': 7, 'neutral': 32, 'entailment': 1}, 'neutral': {'contradiction': 1, 'neutral': 7, 'entailment': 2}}



## Accuracy of labels

In [176]:
def calculate_correct_bias(predictions):
    correct_predictions = predictions['entailment']
    incorrect_predictions = predictions['contradiction'] + predictions['neutral']
    return correct_predictions, incorrect_predictions

def calculate_correct_neutral(predictions):
    correct_predictions = predictions['contradiction'] + predictions['neutral']
    incorrect_predictions = predictions['entailment']
    return correct_predictions, incorrect_predictions

In [175]:
for model, data in results.items():
  rationale_hypothesis_results = data['rationale hypothesis']
  base_hypothesis_results = data['base hypothesis']

  # rationale hypothesis
  bias_correct_predictions, bias_incorrect_predictions = calculate_correct_bias(rationale_hypothesis_results['bias'])
  neutral_correct_predictions, neutral_incorrect_predictions = calculate_correct_neutral(rationale_hypothesis_results['neutral'])
 
  accuracy = ((bias_correct_predictions + neutral_correct_predictions) / (bias_correct_predictions + neutral_correct_predictions + bias_incorrect_predictions +neutral_incorrect_predictions)) * 100

  print(accuracy)
  print()

  # base hypothesis

  bias_correct_predictions, bias_incorrect_predictions = calculate_correct_bias(base_hypothesis_results['bias'])
  neutral_correct_predictions, neutral_incorrect_predictions = calculate_correct_neutral(base_hypothesis_results['neutral'])
 
  accuracy = ((bias_correct_predictions + neutral_correct_predictions) / (bias_correct_predictions + neutral_correct_predictions + bias_incorrect_predictions +neutral_incorrect_predictions)) * 100

  print(accuracy)
  print()
    


    

8.0

68.0

12.0

72.0

26.0

18.0

